<a href="https://colab.research.google.com/github/tocattabrito/analiseDados/blob/main/lista_preliminar_processo_seletivo_simplificado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Esse é uma analise exploratória do RESULTADO PRELIMINAR do PROCESSO SELETIVO SIMPLIFICADO DA SECRETARIA DE EDUCAÇÃO DE
PERNAMBUCO - 2025

#1. PREPARANDO O AMBIENTE NO GOOGLE COLAB


In [ ]:
!pip install pdfplumber
import pdfplumber
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 72.4 MB/s eta 0:00:00


#2. FAZER UPLOAD DO ARQUIVO PDF


In [ ]:
from google.colab import files

uploaded = files.upload() # rode o código para carregar o arquivo em pdf

Saving SEE 2025  - Result2-Prelim-Avaliação Curric_TUTOR.pdf to SEE 2025  - Result2-Prelim-Avaliação Curric_TUTOR.pdf


#3.Ler o conteúdo do PDF

In [ ]:
# Substitua pelo nome do seu arquivo
pdf_path = list(uploaded.keys())[0]


linhas = []

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            linhas.extend(text.split('\n'))  # Quebra por linha

# 4. Criar DataFrame com Pandas


In [ ]:
for linha in linhas[:10]:  # Primeiras 10 linhas
   print(repr(linha))


'PROCESSO SELETIVO SIMPLIFICADO DA SECRETARIA DE EDUCAÇÃO'
'DE PERNAMBUCO - 2025'
'RESULTADO PRELIMINAR'
'TUTOR DA EDUCAÇÃO PROFISSIONAL EAD/POLO'
'O Instituto de Apoio à Fundação Universidade de Pernambuco - IAUPE'
'Concursos, e a Secretaria de Educação de Pernambuco divulgam o resultado'
'preliminar da avaliação da experiência profissional e de títulos.'
'Recife, 27 de Junho de 2025.'
'sexta-feira, 27 de junho de 2025 Página 1 de 398'
'PROCESSO SELETIVO SIMPLIFICADO DA SECRETARIA DE EDUCAÇÃO DE'


In [ ]:
for i, linha in enumerate(linhas[:10]):
    print(f"{i}: {repr(linha)}")


0: 'PROCESSO SELETIVO SIMPLIFICADO DA SECRETARIA DE EDUCAÇÃO'
1: 'DE PERNAMBUCO - 2025'
2: 'RESULTADO PRELIMINAR'
3: 'TUTOR DA EDUCAÇÃO PROFISSIONAL EAD/POLO'
4: 'O Instituto de Apoio à Fundação Universidade de Pernambuco - IAUPE'
5: 'Concursos, e a Secretaria de Educação de Pernambuco divulgam o resultado'
6: 'preliminar da avaliação da experiência profissional e de títulos.'
7: 'Recife, 27 de Junho de 2025.'
8: 'sexta-feira, 27 de junho de 2025 Página 1 de 398'
9: 'PROCESSO SELETIVO SIMPLIFICADO DA SECRETARIA DE EDUCAÇÃO DE'


In [ ]:
import pdfplumber

linhas = []

with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(pdf.pages):
        text = page.extract_text()
        print(f"\n--- Página {i+1} ---\n")
        print(text[:1000])  # imprime os primeiros 1000 caracteres
        if text:
            linhas.extend(text.split('\n'))
        if i >= 2:  # limite para testar só as primeiras 3 páginas
            break



--- Página 1 ---

PROCESSO SELETIVO SIMPLIFICADO DA SECRETARIA DE EDUCAÇÃO
DE PERNAMBUCO - 2025
RESULTADO PRELIMINAR
TUTOR DA EDUCAÇÃO PROFISSIONAL EAD/POLO
O Instituto de Apoio à Fundação Universidade de Pernambuco - IAUPE
Concursos, e a Secretaria de Educação de Pernambuco divulgam o resultado
preliminar da avaliação da experiência profissional e de títulos.
Recife, 27 de Junho de 2025.
sexta-feira, 27 de junho de 2025 Página 1 de 398

--- Página 2 ---

PROCESSO SELETIVO SIMPLIFICADO DA SECRETARIA DE EDUCAÇÃO DE
PERNAMBUCO - 2025
TUTOR DA EDUCAÇÃO PROFISSIONAL EAD/POLO
ATENÇÃO
Os recursos administrativos referente as notas obtidas na avaliação
de experiência profissional e de título, devem ser encaminhados
para o email:
selecaosee2025@iaupe.com.br
No período compreendido entre o dia 28 à 30 de junho de 2025 de
acordo com a errata Nº002 (Anexo VI - Cronograma), devendo ser
informado o número de inscrição, nome do candidato e as razões do
recurso.
sexta-feira, 27 de junho de 2025 Pági

# 5. Converter para DataFrame

In [ ]:
import re
import pandas as pd

dados = []

for linha in linhas:
    match = re.match(r'^(\d{6})\s+(.*)', linha)
    if not match:
        continue

    inscricao = match.group(1)
    restante = match.group(2)

    # Tenta capturar os 4 últimos números com vírgula no final da linha
    pontuacao_match = re.search(r'(\d+,\d+)\s+(\d+,\d+)\s+(\d+,\d+)\s+(\d+,\d+)$', restante)
    if not pontuacao_match:
        continue

    exp_prof = pontuacao_match.group(1)
    ens_medio = pontuacao_match.group(2)
    curso_tec = pontuacao_match.group(3)
    soma = pontuacao_match.group(4)

    nome_pcd = restante[:pontuacao_match.start()].strip()

    if nome_pcd.endswith("PCD"):
        nome = nome_pcd.rsplit("PCD", 1)[0].strip()
        pcd = "PCD"
    else:
        nome = nome_pcd
        pcd = ""

    dados.append([inscricao, nome, pcd, exp_prof, ens_medio, curso_tec, soma])

# Criar DataFrame
df = pd.DataFrame(dados, columns=[
    "INSCRIÇÃO", "NOME DO CANDIDATO", "PCD",
    "Experiência Profissional", "Ensino Médio Completo",
    "Curso Técnico Nível Médio Completo", "SOMA DA PONTUAÇÃO"
])

print("Total de registros extraídos:", len(df))
df.head()


Total de registros extraídos: 17


,INSCRIÇÃO,NOME DO CANDIDATO,PCD,Experiência Profissional,Ensino Médio Completo,Curso Técnico Nível Médio Completo,SOMA DA PONTUAÇÃO
0,116839,AANTONIO EDSON DE LIMA,,"0,0","20,0","0,0","20,0"
1,111481,ABIGAIL JAYNNARA GOMES DA SILVA ALMEIDA,,"0,0","20,0","0,0","20,0"
2,109705,ABIMAEL DE ALMEIDA SIMPLICIO,,"0,0","20,0","20,0","40,0"
3,102654,ABRAÃO SOUZA DA SILVA,,"20,0","20,0","0,0","40,0"
4,110101,ABSON TOME DE SOUZA,,"30,0","20,0","0,0","50,0"


# Perguntas:


## Qual o total de Inscritos com <40:

In [ ]:
import pdfplumber
import pandas as pd

pdf_path = "SEE 2025  - Result2-Prelim-Avaliação Curric_TUTOR.pdf"

linhas = []

with pdfplumber.open(pdf_path) as pdf:
    for i, page in enumerate(pdf.pages):
        if i < 2:  # pula páginas 0 e 1
            continue
        text = page.extract_text()
        if text:
            linhas.extend(text.split('\n'))

# Extração estruturada
import re

dados = []

for linha in linhas:
    match = re.match(r'^(\d{6})\s+(.*)', linha)
    if match:
        partes = linha.split()
        inscricao = partes[0]
        # Extrai as últimas 4 colunas como notas
        notas = partes[-4:]
        try:
            notas_float = [float(n.replace(",", ".")) for n in notas]
        except:
            continue  # pula se der erro
        nome = ' '.join(partes[1:-4])
        dados.append([inscricao, nome] + notas_float)

# Criar DataFrame
df = pd.DataFrame(dados, columns=["INSCRIÇÃO", "NOME", "Experiência Profissional",
                                  "Ensino Médio Completo", "Curso Técnico Nível Médio Completo",
                                  "SOMA DA PONTUAÇÃO"])

# Contar inscritos com nota < 40
total_menor_que_40 = df[df["SOMA DA PONTUAÇÃO"] < 40].shape[0]
print("Total de inscritos com nota < 40:", total_menor_que_40)

total_40 = df[df["SOMA DA PONTUAÇÃO"] == 40].shape[0]
print("Total de inscritos com nota = 40:", total_40)

total_60 = df[df["SOMA DA PONTUAÇÃO"] == 60].shape[0]
print("Total de inscritos com nota = 60:", total_60)

total_80 = df[df["SOMA DA PONTUAÇÃO"] == 80].shape[0]
print("Total de inscritos com nota = 80:", total_80)

total_100 = df[df["SOMA DA PONTUAÇÃO"] == 100].shape[0]
print("Total de inscritos com nota = 100:", total_40)



Total de inscritos com nota < 40: 3217
Total de inscritos com nota = 40: 1186
Total de inscritos com nota = 60: 305
Total de inscritos com nota = 80: 1123
Total de inscritos com nota = 100: 1186


## Total de Inscritos para Tutor:

## Total de inscritos para Tutor:


In [ ]:
total_inscritos = df.shape[0]
print("Total de inscritos:", total_inscritos)


Total de inscritos: 6728


## Percentual por soma da pontuação:

In [ ]:
# Agrupar por nota final e contar quantos tiraram cada uma
contagem = df['SOMA DA PONTUAÇÃO'].value_counts().sort_index()

# Calcular o percentual
percentual = (contagem / df.shape[0]) * 100

# Juntar os dois em um DataFrame
resumo = pd.DataFrame({
    'Quantidade': contagem,
    'Percentual (%)': percentual.round(2)
})

print(resumo)


                   Quantidade  Percentual (%)
SOMA DA PONTUAÇÃO                            
0.0                       279            4.15
10.0                       12            0.18
20.0                     2545           37.83
30.0                      381            5.66
40.0                     1186           17.63
50.0                      345            5.13
60.0                      305            4.53
70.0                      229            3.40
80.0                     1123           16.69
90.0                       48            0.71
100.0                     275            4.09
